## Notebook to recapture knowledge from the KG about NPDIs and compare with NaPDI repo data (in vitro experiments)

* Author: Sanya B Taneja
* Created: 2022-04-06
* Last edited: 2022-04-06

PheKnowLator and MR graphs are combined in the original Path Search notebook and loaded here as the combined graph.

In [208]:
import os
import os.path
import networkx as nx
import json
import urllib
import traceback
from itertools import islice
from rdflib import Graph, URIRef, BNode, Namespace, Literal
from rdflib.namespace import RDF, OWL
from tqdm import tqdm
import json

In [209]:
import hashlib

In [210]:
import pickle
import pandas as pd
import numpy as np

In [211]:
#import pheknowlator kg_utils 
import sys
sys.path.append('../')
from pkt_kg.utils import *

In [212]:
KG_PATH = '/home/sanya/PheKnowLatorv2/resources/knowledge_graphs/'
MR_PATH = '/home/sanya/PheKnowLatorv2/machine_read/output_graphs/'
KG_PATH_MERGED = KG_PATH + 'merged_graphs/'
NodeLabelsFile = KG_PATH_MERGED + 'nodeLabels_merged_20220406.pickle'
KG_NAME_MERGED = 'PheKnowLator_machine_read_merged_instance_based_OWLNETS_20220406.gpickle'

In [213]:
with open(NodeLabelsFile, 'rb') as filep:
    nodeLabels = pickle.load(filep)

In [214]:
def get_graph_stats(kg):
    nodes = nx.number_of_nodes(kg)
    edges = nx.number_of_edges(kg)
    self_loops = nx.number_of_selfloops(kg)

    print('There are {} nodes, {} edges, and {} self-loop(s)'.format(nodes, edges, self_loops))
    # get degree information
    avg_degree = float(edges) / nodes
    print('The Average Degree is {}'.format(avg_degree))
    
    print('Nodes with highest degree:')
    n_deg = sorted([(str(x[0]), x[1]) for x in  kg.degree], key=lambda x: x[1], reverse=1)[:6]

    for x in n_deg:
        print('Label: {}'.format(nodeLabels[x[0]]))
        print('{} (degree={})'.format(x[0], x[1]))
    # get network density
    density = nx.density(kg)

    print('The density of the graph is: {}'.format(density))

In [8]:
##READ MERGED GRAPH
nx_graph = nx.read_gpickle(KG_PATH_MERGED+KG_NAME_MERGED)


In [215]:
get_graph_stats(nx_graph)

There are 745763 nodes, 7251937 edges, and 562 self-loop(s)
The Average Degree is 9.72418449292872
Nodes with highest degree:
Label: transcript
http://purl.obolibrary.org/obo/SO_0000673 (degree=190858)
Label: SNV
http://purl.obolibrary.org/obo/SO_0001483 (degree=121020)
Label: Homo sapiens
http://purl.obolibrary.org/obo/NCBITaxon_9606 (degree=116704)
Label: protein_coding_gene
http://purl.obolibrary.org/obo/SO_0001217 (degree=105046)
Label: testis
http://purl.obolibrary.org/obo/UBERON_0000473 (degree=43818)
Label: lncRNA_with_retained_intron
http://purl.obolibrary.org/obo/SO_0002113 (degree=29340)
The density of the graph is: 1.3039259834811533e-05


In [9]:
DIR_OUT = '/home/sanya/PheKnowLatorv2/output_files/'
obo = Namespace('http://purl.obolibrary.org/obo/')
napdi = Namespace('http://napdi.org/napdi_srs_imports:')

In [10]:
nodeLabels['http://purl.obolibrary.org/obo/CHEBI_4806']

'(-)-epigallocatechin 3-gallate'

In [11]:
##Path between midazolam (CHEBI_) and 1'-hydroxymidazolam (CHEBI_145330)
##Link to in vitro inhibition of mitragynine and midazolam - interaction occurs in intestine - pathway?

In [216]:
import KG_path_searches

In [165]:
##read in pickle dictionary with obo identifiers
import pickle
with open(KG_PATH+'strToOBOdict.pickle','rb') as filep:
    obodict = pickle.load(filep)
len(obodict)

40

In [17]:
##Kratom with -- We present a case of a patient taking 150 mg venlafaxine (CYP2D6/3A substrate), 300 mg quetiapine (CYP3A substrate), 
#and a high amount of kratom (∼90 g) daily. The patient presented to the emergency department with serotonin syndrome and 
#corrected electrocardiogram abnormalities that may have been secondary to supratherapeutic exposure to venlafaxine and/or quetiapine.

In [27]:
##Green tea and nadolol
##Green tea and raloxifene

raloxifene = obo.CHEBI_8772
nodeLabels['http://purl.obolibrary.org/obo/CHEBI_8772']

'raloxifene'

In [41]:
nx_graph.get_edge_data(raloxifene, obo.CHEBI_50837)

{rdflib.term.URIRef('http://purl.obolibrary.org/obo/RO_0000087'): {'predicate_key': '6d3c663b6832d25f9f902311a4a222d1',
  'weight': 0.0}}

In [36]:
nx_graph.get_edge_data(raloxifene, obo.PR_P78381)[URIRef('http://purl.obolibrary.org/obo/RO_0002449')]

{'predicate_key': '6c12966e2e06c036d23c9289a8499790',
 'weight': 0.0,
 'pmid': '18371987',
 'timestamp': '2008 Apr 23',
 'source_graph': 'machine_read',
 'belief': 0.65}

In [13]:
def get_neighbors(nx_graph, node):
    neighs = nx.neighbors(nx_graph, node)
    neighs_list = []
    for item in neighs:
        nodestr = nodeLabels[str(node)]
        neighbor = nodeLabels[str(item)]
        edge_result = nx_graph.get_edge_data(node, item)
        edge_keys = list(edge_result.keys())
        for edge in edge_keys:
            edgestr = nodeLabels[str(edge)]
            edgevalue = edge_result[edge]
            result = [nodestr, edgestr, neighbor]
            if 'source_graph' in edgevalue:
                result.append(edgevalue['source_graph'])
            if 'timestamp' in edgevalue:
                result.append(edgevalue['timestamp'])
            neighs_list.append(result)
    return neighs_list

In [14]:
inhibit_preds = [obo.RO_0002449, obo.RO_0002212, obo.RO_0002599, obo.RO_0011010]
active_preds = [obo.RO_0002448, obo.RO_0002213, obo.RO_0011002, obo.RO_0002596, obo.RO_0002598, obo.RO_0011009, obo.RO_0002566]
interact_preds = [obo.DIDEO_00000041, obo.RO_0002434, obo.RO_0002436, obo.RO_0002020, obo.RO_0003000, obo.RO_0000085, 
                  URIRef('http://purl.obolibrary.org/obo/chebi#has_functional_parent')]
protein_preds = []

In [193]:
def get_neighbors_filtered(nx_graph, node):
    neighs = nx.neighbors(nx_graph, node)
    neighs_list = []
    nodestr = ''
    neighbor = ''
    for item in neighs:
        if str(node) in nodeLabels:
            nodestr = nodeLabels[str(node)]
        if str(item) in nodeLabels:
            neighbor = nodeLabels[str(item)]
        edge_result = nx_graph.get_edge_data(node, item)
        edge_keys = list(edge_result.keys())
        for edge in edge_keys:
            if edge in inhibit_preds or edge in active_preds or edge in interact_preds:
                edgestr = nodeLabels[str(edge)]
                edgevalue = edge_result[edge]
                result = [nodestr, edgestr, neighbor]
                if 'source_graph' in edgevalue:
                    result.append(edgevalue['source_graph'])
                if 'timestamp' in edgevalue:
                    result.append(edgevalue['timestamp'])
                neighs_list.append(result)
    return neighs_list

In [19]:
list(nx_graph.get_edge_data(obodict['mitragynine'][0], obodict['CYP3A4'][0]).keys())

[rdflib.term.URIRef('http://purl.obolibrary.org/obo/RO_0011009'),
 rdflib.term.URIRef('http://purl.obolibrary.org/obo/RO_0002449'),
 rdflib.term.URIRef('http://purl.obolibrary.org/obo/RO_0002448'),
 rdflib.term.URIRef('http://purl.obolibrary.org/obo/RO_0002434')]

In [20]:
get_neighbors(nx_graph, raloxifene)

NameError: name 'raloxifene' is not defined

In [21]:
##read in negated predications
df_neg = pd.read_csv('../machine_read/semrep_data/semrep_negation_predications_mapped_20220407.csv')
df_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1796 entries, 0 to 1795
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   pmid           1796 non-null   int64  
 1   subject_cui    1755 non-null   object 
 2   subject_type   1796 non-null   object 
 3   object_cui     1719 non-null   object 
 4   object_type    1796 non-null   object 
 5   year           1790 non-null   float64
 6   subject_name   1755 non-null   object 
 7   predicate      1796 non-null   object 
 8   object_name    1719 non-null   object 
 9   source_text    1796 non-null   object 
 10  Unnamed: 10    0 non-null      float64
 11  predicate_obo  0 non-null      float64
 12  subject_obo    1194 non-null   object 
 13  object_obo     831 non-null    object 
dtypes: float64(3), int64(1), object(10)
memory usage: 196.6+ KB


## Kratom - Mechanistic Exploration Part 1

In [22]:
nx_graph.__getitem__(obodict['kratom'][0])

AdjacencyView({rdflib.term.URIRef('http://purl.obolibrary.org/obo/PO_0025131'): {rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf'): {'predicate_key': '958521878a132c4bfad53e7ed70ac7f3', 'weight': 0.0}}, rdflib.term.URIRef('http://purl.obolibrary.org/obo/CHEBI_6956'): {rdflib.term.URIRef('http://purl.obolibrary.org/obo/RO_0002180'): {'predicate_key': '92919f4d7a39d7db676c369dd45a1c1c', 'weight': 0.0}}, rdflib.term.URIRef('http://napdi.org/napdi_srs_imports:mitragyna_speciosa_whole'): {rdflib.term.URIRef('http://purl.obolibrary.org/obo/BFO_0000050'): {'predicate_key': '2aaba9f764274816c387a21ce0acb793', 'weight': 0.0}}, rdflib.term.URIRef('http://napdi.org/napdi_srs_imports:7_hydroxy_mitragynine'): {rdflib.term.URIRef('http://purl.obolibrary.org/obo/RO_0002180'): {'predicate_key': '3ebb29585d901f255e32b08d4a6a61a2', 'weight': 0.0}}})

In [23]:
mit_neighbors = get_neighbors(nx_graph, obodict['mitragynine'][0])
len(mit_neighbors)

121

In [24]:
mit_neighbors_filt = get_neighbors_filtered(nx_graph, obodict['mitragynine'][0])
len(mit_neighbors_filt)

98

In [166]:
obodict['PGP']

[rdflib.term.URIRef('http://purl.obolibrary.org/obo/PR_P000001891'),
 rdflib.term.URIRef('http://purl.obolibrary.org/obo/PR_P08183'),
 rdflib.term.URIRef('http://purl.obolibrary.org/obo/GO_0008559'),
 rdflib.term.URIRef('http://purl.obolibrary.org/obo/PR_000012619')]

In [26]:
edges = nx_graph.get_edge_data(obodict['mitragynine'][0], obodict['CYP3A4'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['mitragynine'][0], obodict['CYP3A4'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

{'entity_type': 'RELATIONS', 'label': 'IncreaseAmount'}
{'predicate_key': '04792bbeb7346991d4206904a9cd8164', 'weight': 0.0, 'pmid': '23274770', 'timestamp': '2013 Mar', 'source_graph': 'machine_read', 'belief': 0.65}
directly negatively regulates activity of
{'predicate_key': '35db33b594d5b68623d10e9a02d4c781', 'weight': 0.0, 'pmid': '24174816', 'timestamp': '2013 Oct', 'source_graph': 'machine_read', 'belief': 0.86}
{'entity_type': 'RELATIONS', 'label': 'Activation'}
{'predicate_key': '53929eb7232ad497ae25cf0f8b3f31b7', 'weight': 0.0, 'pmid': '23274770', 'timestamp': '2013 Mar', 'source_graph': 'machine_read', 'belief': 0.65}
interacts with
{'predicate_key': 'd7cc44c1009eda758102414ff4eb1e6a', 'weight': 0.0}


In [27]:
edges = nx_graph.get_edge_data(obodict['mitragynine'][0], obodict['CYP2D6'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['mitragynine'][0], obodict['CYP2D6'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

directly negatively regulates activity of
{'predicate_key': '95f12736d1211905a358aef7ffba851b', 'weight': 0.0, 'pmid': '29071751', 'timestamp': '2017.0', 'source_graph': 'machine_read', 'belief': 0.8}
interacts with
{'predicate_key': '21e30bf3c3f213311b2c497b8b198e06', 'weight': 0.0}
directly negatively regulates activity of
{'predicate_key': '788f0f936532db4f71a8a647692cab3e', 'weight': 0.0, 'pmid': '24174816', 'timestamp': '2013.0', 'source_graph': 'machine_read', 'belief': 0.8}
positively regulates
{'predicate_key': '40e4364894f99e7cb08fd0b20b6ace3f', 'weight': 0.0, 'pmid': '23274770', 'timestamp': '2013.0', 'source_graph': 'machine_read', 'belief': 0.8}
interacts with
{'predicate_key': '64c44bee7a60dc54ada041615f0b0371', 'weight': 0.0, 'pmid': '30547698', 'timestamp': '2019.0', 'source_graph': 'machine_read_INF', 'belief': 0.8}


In [28]:
edges = nx_graph.get_edge_data(obodict['mitragynine'][0], obodict['CYP2C19'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['mitragynine'][0], obodict['CYP2C19'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

interacts with
{'predicate_key': 'a2f1439cda4a0a2da0b67705d23facce', 'weight': 0.0}


In [29]:
edges = nx_graph.get_edge_data(obodict['mitragynine'][0], obodict['CYP2C9'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['mitragynine'][0], obodict['CYP2C9'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

directly negatively regulates activity of
{'predicate_key': '5044e2ac537ebe9f92096235427026f3', 'weight': 0.0, 'pmid': '24174816', 'timestamp': '2013 Oct', 'source_graph': 'machine_read', 'belief': 0.86}
directly negatively regulates activity of
{'predicate_key': 'e593974d2818ab169fbbe255d52646fd', 'weight': 0.0, 'pmid': '24174816', 'timestamp': '2013.0', 'source_graph': 'machine_read', 'belief': 0.8}


In [30]:
edges = nx_graph.get_edge_data(obodict['mitragynine'][0], obodict['CYP1A2'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['mitragynine'][0], obodict['CYP1A2'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

{'entity_type': 'RELATIONS', 'label': 'IncreaseAmount'}
{'predicate_key': 'd7351c8523bd7cbf34d43b6a412a0bbe', 'weight': 0.0, 'pmid': '23274770', 'timestamp': '2013 Mar', 'source_graph': 'machine_read', 'belief': 0.86}
{'entity_type': 'RELATIONS', 'label': 'Activation'}
{'predicate_key': '77c46a33e2f9a70ee383dfd247785a23', 'weight': 0.0, 'pmid': '29071751', 'timestamp': '2017 Dec', 'source_graph': 'machine_read', 'belief': 0.86}
interacts with
{'predicate_key': '0553e1d0f9b0b6920c61bbcbc1c04db8', 'weight': 0.0}
interacts with
{'predicate_key': '572622b80f7e7965769037a2c42ec144', 'weight': 0.0, 'pmid': '23274770', 'timestamp': '2013.0', 'source_graph': 'machine_read_INF', 'belief': 0.8}
positively regulates
{'predicate_key': '1b698e5a28f61543a1014e3adf0998c2', 'weight': 0.0, 'pmid': '23274770', 'timestamp': '2013.0', 'source_graph': 'machine_read_INF', 'belief': 0.8}


In [31]:
edges = nx_graph.get_edge_data(obodict['mitragynine'][0], obodict['CYP2C8'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['mitragynine'][0], obodict['CYP2C8'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

In [32]:
edges = nx_graph.get_edge_data(obodict['mitragynine'][0], obodict['CYP3A5'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['mitragynine'][0], obodict['CYP3A5'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

In [129]:
edges = nx_graph.get_edge_data(obodict['mitragynine'][0], obodict['UGT'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['mitragynine'][0], obodict['UGT'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['mitragynine'][0], obodict['UGT'][2])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

In [34]:
edges = nx_graph.get_edge_data(obodict['mitragynine'][0], obodict['UGT1A1'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['mitragynine'][0], obodict['UGT1A1'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

directly negatively regulates activity of
{'predicate_key': '77b223d7adda942f0f53c2d6c25717cb', 'weight': 0.0, 'pmid': '26692748', 'timestamp': '2014.0', 'source_graph': 'machine_read', 'belief': 0.8}


In [35]:
edges = nx_graph.get_edge_data(obodict['mitragynine'][0], obodict['UGT2B7'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['mitragynine'][0], obodict['UGT2B7'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

directly negatively regulates activity of
{'predicate_key': '6825c6627114a1d163b63d7f024d0289', 'weight': 0.0, 'pmid': '26692748', 'timestamp': '2014.0', 'source_graph': 'machine_read', 'belief': 0.8}


In [167]:
edges = nx_graph.get_edge_data(obodict['mitragynine'][0], obodict['PGP'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['mitragynine'][0], obodict['PGP'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['mitragynine'][0], obodict['PGP'][2])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['mitragynine'][0], obodict['PGP'][3])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

{'entity_type': 'RELATIONS', 'label': 'DecreaseAmount'}
{'predicate_key': '7b16891a8684bc49ca03f724b93f2266', 'weight': 0.0, 'pmid': '30604191', 'timestamp': '2019 Apr', 'source_graph': 'machine_read', 'belief': 0.9419}
directly negatively regulates activity of
{'predicate_key': '1d2db53249d0e93cedb141dc363e65bb', 'weight': 0.0, 'pmid': '30604191', 'timestamp': '2019 Apr', 'source_graph': 'machine_read', 'belief': 0.65}
directly negatively regulates activity of
{'predicate_key': '3cd523ea48bad379fe3031cf46d9747f', 'weight': 0.0, 'pmid': '34003732', 'timestamp': '2021.0', 'source_graph': 'machine_read', 'belief': 0.8}


In [153]:
edges = nx_graph.get_edge_data(obodict['mitragynine'][0], obodict['glycoprotein'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['mitragynine'][0], obodict['glycoprotein'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

In [44]:
get_neighbors(nx_graph, obodict['hydroxy_mitragynine'][0])

[['7-hydroxy-mitragynine', 'subClassOf', 'chemical entity'],
 ['7-hydroxy-mitragynine', 'has role', 'metabolite'],
 ['7-hydroxy-mitragynine', 'has functional parent', 'Mitragynine']]

In [54]:
get_neighbors(nx_graph, obodict['hydroxy_mitragynine'][1])

[[{'entity_type': 'NODES', 'label': '7-hydroxymitragynine', 'cui': 'C1435927'},
  'directly negatively regulates activity of',
  '4-methylumbelliferone beta-D-glucuronide',
  'machine_read',
  '2014.0'],
 [{'entity_type': 'NODES', 'label': '7-hydroxymitragynine', 'cui': 'C1435927'},
  'interacts with',
  '4-methylumbelliferone beta-D-glucuronide',
  'machine_read_INF',
  '2014.0'],
 [{'entity_type': 'NODES', 'label': '7-hydroxymitragynine', 'cui': 'C1435927'},
  'existence overlaps',
  'methanol',
  'machine_read',
  '2014.0'],
 [{'entity_type': 'NODES', 'label': '7-hydroxymitragynine', 'cui': 'C1435927'},
  'directly negatively regulates activity of',
  'Mitragynine',
  'machine_read',
  '2014.0'],
 [{'entity_type': 'NODES', 'label': '7-hydroxymitragynine', 'cui': 'C1435927'},
  'directly negatively regulates activity of',
  'UDP-glucuronosyltransferase 2B7',
  'machine_read',
  '2014.0'],
 [{'entity_type': 'NODES', 'label': '7-hydroxymitragynine', 'cui': 'C1435927'},
  'existence ove

In [131]:
edges = nx_graph.get_edge_data(obodict['hydroxy_mitragynine'][1], obodict['UGT'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['hydroxy_mitragynine'][1], obodict['UGT'][2])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

In [58]:
edges = nx_graph.get_edge_data(obodict['hydroxy_mitragynine'][0], obodict['mitragynine'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['hydroxy_mitragynine'][1], obodict['mitragynine'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

has functional parent
{'predicate_key': '500db2060a025b3ab27fcc2602acb6c2', 'weight': 0.0}
directly negatively regulates activity of
{'predicate_key': 'e8de6bae777ac5a1078eb563887b72a8', 'weight': 0.0, 'pmid': '26692748', 'timestamp': '2014.0', 'source_graph': 'machine_read', 'belief': 0.8}


In [42]:
nodeLabels['http://purl.obolibrary.org/obo/CHEBI_25212']

'metabolite'

In [25]:
df_neg.head()

,pmid,subject_cui,subject_type,object_cui,object_type,year,subject_name,predicate,object_name,source_text,Unnamed: 10,predicate_obo,subject_obo,object_obo
0,19353999,C0059438,"orch,phsu",C1332826,"gngm,aapp",2009.0,epigallocatechin gallate,neg_interacts_with,CYP1A2 gene,19353999_ascii.txt.tx.1 A recent report has fo...,NaN,NaN,napdi_srs_imports:epigallocatechin_gallate,PR_000006102
1,12659723,C0007404,"orch,phsu",C1420188,"gngm,aapp",2003.0,Catechin,neg_interacts_with,SLC35A2 gene,12659723_ascii.txt.tx.2 Lhoste et al./ Food an...,NaN,NaN,CHEBI_23053,PR_000015090
2,12659723,C0022203,npop,C0011135,orgf,2003.0,Isomerism (chemical phenomenon),neg_affects,Defecation,"12659723_ascii.txt.tx.2 Other polyphenols, e.g...",NaN,NaN,NaN,NaN
3,32645157,C0041912,dsyn,C1154978,orgf,2020.0,Upper Respiratory Infections,neg_affects,cold acclimation,32645157_ascii.txt.tx.1 After undergoing cold ...,NaN,NaN,MONDO_0024355,NaN
4,32645157,C1167041,celc,C0120285,"aapp,gngm,irda",2020.0,vacuolar membrane,neg_location_of,Green Fluorescent Proteins,32645157_ascii.txt.tx.1 Our results showed tha...,NaN,NaN,NaN,NaN


In [39]:
df_neg.loc[df_neg['subject_name'] == 'mitragynine']

,pmid,subject_cui,subject_type,object_cui,object_type,year,subject_name,predicate,object_name,source_text,Unnamed: 10,predicate_obo,subject_obo,object_obo
1679,27260674,C0066619,"orch,phsu",C1416572,"gngm,aapp",2016.0,mitragynine,neg_stimulates,KCNH2 gene,27260674_ascii.txt.tx.2 Mitragynine did not in...,NaN,NaN,CHEBI_6956,PR_000000791
1717,21876481,C0066619,"orch,phsu",C0026549,"orch,phsu",2011.0,mitragynine,neg_same_as,Morphine,21876481_ascii.txt.tx.2 Each point was Mitr...,NaN,NaN,CHEBI_6956,CHEBI_17303
1746,16107269,C0066619,"orch,phsu",C0034801,"aapp,gngm,rcpt",2005.0,mitragynine,neg_interacts_with,Opioid Receptor,16107269_ascii.txt.tx.1 This does not necessar...,NaN,NaN,CHEBI_6956,NaN
1761,23274770,C0066619,"orch,phsu",NaN,"gngm,aapp",2013.0,mitragynine,neg_interacts_with,NaN,"23274770_ascii.txt.tx.1 Interestingly, and unl...",NaN,NaN,CHEBI_6956,NaN
1762,23274770,C0066619,"orch,phsu",C1515670,genf,2013.0,mitragynine,neg_affects,mRNA Expression,23274770_ascii.txt.tx.1 Mitragynine does not a...,NaN,NaN,CHEBI_6956,NaN
1763,23274770,C0066619,"orch,phsu",C0544357,phsu,2013.0,mitragynine,neg_coexists_with,human plasma,"23274770_ascii.txt.tx.1 Logically, concentrati...",NaN,NaN,CHEBI_6956,NaN


In [46]:
##7-hydroxy-mitragynine
df_neg.loc[df_neg['object_obo'] == 'CHEBI_180536']

,pmid,subject_cui,subject_type,object_cui,object_type,year,subject_name,predicate,object_name,source_text,Unnamed: 10,predicate_obo,subject_obo,object_obo


In [77]:
len(get_neighbors_filtered(nx_graph, obodict['mitragynine'][0]))

98

## Green Tea - Mechanistic Exploration Part 1

In [74]:
##read through - looks like useful information and a lot from machine read and inferred 
len(get_neighbors_filtered(nx_graph, obodict['greentea'][0]))

87

In [79]:
get_neighbors_filtered(nx_graph, obodict['hydroxy_mitragynine'][1])

[[{'entity_type': 'NODES', 'label': '7-hydroxymitragynine', 'cui': 'C1435927'},
  'directly negatively regulates activity of',
  '4-methylumbelliferone beta-D-glucuronide',
  'machine_read',
  '2014.0'],
 [{'entity_type': 'NODES', 'label': '7-hydroxymitragynine', 'cui': 'C1435927'},
  'interacts with',
  '4-methylumbelliferone beta-D-glucuronide',
  'machine_read_INF',
  '2014.0'],
 [{'entity_type': 'NODES', 'label': '7-hydroxymitragynine', 'cui': 'C1435927'},
  'directly negatively regulates activity of',
  'Mitragynine',
  'machine_read',
  '2014.0'],
 [{'entity_type': 'NODES', 'label': '7-hydroxymitragynine', 'cui': 'C1435927'},
  'directly negatively regulates activity of',
  'UDP-glucuronosyltransferase 2B7',
  'machine_read',
  '2014.0']]

In [149]:
df_neg.head()

,pmid,subject_cui,subject_type,object_cui,object_type,year,subject_name,predicate,object_name,source_text,Unnamed: 10,predicate_obo,subject_obo,object_obo
0,19353999,C0059438,"orch,phsu",C1332826,"gngm,aapp",2009.0,epigallocatechin gallate,neg_interacts_with,CYP1A2 gene,19353999_ascii.txt.tx.1 A recent report has fo...,NaN,NaN,napdi_srs_imports:epigallocatechin_gallate,PR_000006102
1,12659723,C0007404,"orch,phsu",C1420188,"gngm,aapp",2003.0,Catechin,neg_interacts_with,SLC35A2 gene,12659723_ascii.txt.tx.2 Lhoste et al./ Food an...,NaN,NaN,CHEBI_23053,PR_000015090
2,12659723,C0022203,npop,C0011135,orgf,2003.0,Isomerism (chemical phenomenon),neg_affects,Defecation,"12659723_ascii.txt.tx.2 Other polyphenols, e.g...",NaN,NaN,NaN,NaN
3,32645157,C0041912,dsyn,C1154978,orgf,2020.0,Upper Respiratory Infections,neg_affects,cold acclimation,32645157_ascii.txt.tx.1 After undergoing cold ...,NaN,NaN,MONDO_0024355,NaN
4,32645157,C1167041,celc,C0120285,"aapp,gngm,irda",2020.0,vacuolar membrane,neg_location_of,Green Fluorescent Proteins,32645157_ascii.txt.tx.1 Our results showed tha...,NaN,NaN,NaN,NaN


#### Green tea leaf

In [80]:
##there is an edge with glycoprotein (CHEBI_)

edges = nx_graph.get_edge_data(obodict['greentea'][0], obo.CHEBI_17089)
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['greentea'][0], obo.PR_000037069)
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

directly negatively regulates activity of
{'predicate_key': '330824948c6308aba5c71305c24a46ab', 'weight': 0.0, 'pmid': '15319329', 'timestamp': '2004.0', 'source_graph': 'machine_read', 'belief': 0.8}


In [132]:
edges = nx_graph.get_edge_data(obodict['greentea'][0], obodict['UGT'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['greentea'][0], obodict['UGT'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['greentea'][0], obodict['UGT'][2])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

directly negatively regulates activity of
{'predicate_key': '2da8cfaf56cc552a738b5007546fb60b', 'weight': 0.0, 'pmid': '29467215', 'timestamp': '2018.0', 'source_graph': 'machine_read', 'belief': 0.8}


In [169]:
edges = nx_graph.get_edge_data(obodict['greentea'][0], obodict['PGP'][3])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

In [174]:
df_neg.loc[df_neg['subject_obo'] == 'napdi_srs_imports:camellia_sinensis_leaf']

,pmid,subject_cui,subject_type,object_cui,object_type,year,subject_name,predicate,object_name,source_text,Unnamed: 10,predicate_obo,subject_obo,object_obo
13,20471964,C1704263,"orch,phsu",C0030012,moft,2010.0,Green Tea Extract,neg_affects,Oxidation-Reduction,20471964_ascii.txt.tx.3 Catechins are also kno...,NaN,NaN,napdi_srs_imports:camellia_sinensis_leaf,NaN
103,25418056,C0039400,food,C0031327,phsf,2015.0,Tea,neg_affects,Drug Kinetics,25418056_ascii.txt.tx.1 With the continuing ef...,NaN,NaN,napdi_srs_imports:camellia_sinensis_leaf,NaN
205,29570984,C0039400,food,C0017638,neop,2018.0,Tea,neg_predisposes,Glioma,29570984_ascii.txt.tx.1 335 glioma cases NIH-A...,NaN,NaN,napdi_srs_imports:camellia_sinensis_leaf,NaN
206,29570984,C0039400,food,C0017638,neop,2018.0,Tea,neg_predisposes,Glioma,29570984_ascii.txt.tx.3 With the first quantil...,NaN,NaN,napdi_srs_imports:camellia_sinensis_leaf,NaN
420,31100973,C0039400,food,C0013470,orgf,2019.0,Tea,neg_affects,Eating,"31100973_ascii.txt.tx.3 However, supplementati...",NaN,NaN,napdi_srs_imports:camellia_sinensis_leaf,NaN
421,31100973,C0039400,food,C0020615,dsyn,2019.0,Tea,neg_causes,Hypoglycemia,31100973_ascii.txt.tx.2 Green tea was then adm...,NaN,NaN,napdi_srs_imports:camellia_sinensis_leaf,NaN
422,31100973,C0039400,food,C0178665,phsf,2019.0,Tea,neg_affects,glucose tolerance,31100973_ascii.txt.tx.1 Since the blood glucos...,NaN,NaN,napdi_srs_imports:camellia_sinensis_leaf,NaN
503,23106150,C0039400,food,C0013470,orgf,2012.0,Tea,neg_affects,Eating,23106150_ascii.txt.tx.3 During the feeding per...,NaN,NaN,napdi_srs_imports:camellia_sinensis_leaf,NaN
514,24656388,C1704263,"orch,phsu",C2698419,"gngm,aapp",2014.0,Green Tea Extract,neg_interacts_with,NR0B1 wt Allele,24656388_ascii.txt.tx.1 / Journal of Nutrition...,NaN,NaN,napdi_srs_imports:camellia_sinensis_leaf,PR_000011390
549,19170155,C1704263,"orch,phsu",C0002333,"orch,phsu",2009.0,Green Tea Extract,neg_interacts_with,Alprazolam,19170155_ascii.txt.tx.1 One pharmacokinetic st...,NaN,NaN,napdi_srs_imports:camellia_sinensis_leaf,CHEBI_2611


In [126]:
get_neighbors_filtered(nx_graph, obodict['greentea'][0])

[['Camellia_sinensis_leaf',
  'positively regulates',
  'cytochrome P450 1A2',
  'machine_read',
  '2005.0'],
 ['Camellia_sinensis_leaf',
  'directly negatively regulates activity of',
  'cytochrome P450 1A2',
  'machine_read',
  '2009.0'],
 ['Camellia_sinensis_leaf',
  'interacts with',
  'cytochrome P450 1A2',
  'machine_read_INF',
  '2003.0'],
 ['Camellia_sinensis_leaf',
  'interacts with',
  'ion',
  'machine_read_INF',
  '2013.0'],
 ['Camellia_sinensis_leaf',
  'directly negatively regulates activity of',
  'N-methyl-4-phenylpyridinium',
  'machine_read',
  '2006.0'],
 ['Camellia_sinensis_leaf',
  'positively regulates',
  'insulin receptor substrate 2 (human)',
  'machine_read_INF',
  '2013.0'],
 ['Camellia_sinensis_leaf',
  {'entity_type': 'RELATIONS', 'label': 'affects'},
  'activation of protein kinase C activity',
  'machine_read',
  '2006.0'],
 ['Camellia_sinensis_leaf',
  'directly negatively regulates activity of',
  'glycoprotein',
  'machine_read',
  '2004.0'],
 ['Camell

#### EGCG

In [81]:
edges = nx_graph.get_edge_data(obodict['EGCG'][0], obodict['CYP3A4'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['EGCG'][1], obodict['CYP3A4'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['EGCG'][0], obodict['CYP3A4'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['EGCG'][1], obodict['CYP3A4'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

positively regulates
{'predicate_key': '9bd25ce60b782ba99fadae48b130a80c', 'weight': 0.0, 'pmid': '33387731', 'timestamp': '2021.0', 'source_graph': 'machine_read', 'belief': 0.8}


In [82]:
edges = nx_graph.get_edge_data(obodict['EGCG'][0], obodict['CYP2C19'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['EGCG'][1], obodict['CYP2C19'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['EGCG'][0], obodict['CYP2C19'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['EGCG'][1], obodict['CYP2C19'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

directly negatively regulates activity of
{'predicate_key': '6c8b1cda2d2b3a1275555285284e3356', 'weight': 0.0, 'pmid': '29275296', 'timestamp': '2018 Jan-Feb', 'source_graph': 'machine_read', 'belief': 0.86}


In [92]:
edges = nx_graph.get_edge_data(obodict['EGCG'][0], obodict['CYP2E1'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['EGCG'][1], obodict['CYP2E1'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['EGCG'][0], obodict['CYP2E1'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['EGCG'][1], obodict['CYP2E1'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

positively regulates
{'predicate_key': '21515ea83ceaa7c5cbf4c5ec518fd8c1', 'weight': 0.0, 'pmid': '15629189', 'timestamp': '2005.0', 'source_graph': 'machine_read', 'belief': 0.8}
directly negatively regulates activity of
{'predicate_key': 'f098cb7f799d0373b0086f70b29998ef', 'weight': 0.0, 'pmid': '17490800', 'timestamp': '2007.0', 'source_graph': 'machine_read', 'belief': 0.8}
interacts with
{'predicate_key': 'cb2be9eab32ac6ccde3ad5a8202b186c', 'weight': 0.0, 'pmid': '17490800', 'timestamp': '2007.0', 'source_graph': 'machine_read_INF', 'belief': 0.8}


In [93]:
edges = nx_graph.get_edge_data(obodict['EGCG'][0], obodict['UGT'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['EGCG'][1], obodict['UGT'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])


In [99]:
edges = nx_graph.get_edge_data(obodict['EGCG'][0], obodict['UGT1A8'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['EGCG'][1], obodict['UGT1A8'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['EGCG'][0], obodict['UGT1A8'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['EGCG'][1], obodict['UGT1A8'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

{'entity_type': 'RELATIONS', 'label': 'IncreaseAmount'}
{'predicate_key': '859ab4ce500a5fc5cf7c1090e1b99e54', 'weight': 0.0, 'pmid': '17657175', 'timestamp': '2007', 'source_graph': 'machine_read', 'belief': 0.65}


In [109]:
edges = nx_graph.get_edge_data(obodict['EGCG'][0], obodict['OATP1B3'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['EGCG'][1], obodict['OATP1B3'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['EGCG'][0], obodict['OATP1B3'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['EGCG'][1], obodict['OATP1B3'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

directly negatively regulates activity of
{'predicate_key': '7ffd5f6ef41efd68711693dde83633d2', 'weight': 0.0, 'pmid': '21278283', 'timestamp': '2011 May', 'source_graph': 'machine_read', 'belief': 0.65}
{'entity_type': 'RELATIONS', 'label': 'Activation'}
{'predicate_key': '6578c214afe21ae7a260029628026e6e', 'weight': 0.0, 'pmid': '23327877', 'timestamp': '2013 Mar 22', 'source_graph': 'machine_read', 'belief': 0.949999985651093}
directly negatively regulates activity of
{'predicate_key': 'd92bbad7a657363b867659e6aa32b80b', 'weight': 0.0, 'pmid': '28118673', 'timestamp': '2017.0', 'source_graph': 'machine_read', 'belief': 0.8}
positively regulates
{'predicate_key': '3a53c5f9f3dbef1cbf4e2fe575ae8561', 'weight': 0.0, 'pmid': '28118673', 'timestamp': '2017.0', 'source_graph': 'machine_read', 'belief': 0.8}
interacts with
{'predicate_key': '7888647f27c3d7660ca8662b5d453ca4', 'weight': 0.0, 'pmid': '21278283', 'timestamp': '2011.0', 'source_graph': 'machine_read_INF', 'belief': 0.8}


In [107]:

edges = nx_graph.get_edge_data(obodict['EGCG'][1], obo.CHEBI_17089)
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['EGCG'][1], obo.PR_000037069)
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

interacts with
{'predicate_key': '90a9ba250c26eb2fff346a01179043ad', 'weight': 0.0, 'pmid': '26460146', 'timestamp': '2015.0', 'source_graph': 'machine_read_INF', 'belief': 0.8}


In [134]:
edges = nx_graph.get_edge_data(obodict['EGCG'][1], obodict['UGT'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['EGCG'][1], obodict['UGT'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['EGCG'][1], obodict['UGT'][2])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

directly negatively regulates activity of
{'predicate_key': '54bcb546faa1587a70554e63dbc07307', 'weight': 0.0, 'pmid': '29467215', 'timestamp': '2018.0', 'source_graph': 'machine_read', 'belief': 0.8}
positively regulates
{'predicate_key': '3cb3d030cf33323c53bb1ab022d4c8b5', 'weight': 0.0, 'source_graph': 'machine_read_INF'}


In [162]:
#how to get through these???
len(get_neighbors_filtered(nx_graph, obodict['EGCG'][0]))

805

In [170]:
edges = nx_graph.get_edge_data(obodict['EGCG'][0], obodict['PGP'][3])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['EGCG'][1], obodict['PGP'][3])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

existence overlaps
{'predicate_key': 'cc95487a2b0a7cce0323bafea41baf3b', 'weight': 0.0, 'pmid': '14511674', 'timestamp': '2003.0', 'source_graph': 'machine_read', 'belief': 0.8}
directly negatively regulates activity of
{'predicate_key': 'b3889a0b7b0cba37bc2e82841d41d1bb', 'weight': 0.0, 'pmid': '12499265', 'timestamp': '2002.0', 'source_graph': 'machine_read', 'belief': 0.8}


In [163]:
#216 results - saved to file

df_neg.loc[df_neg['subject_obo'] == 'napdi_srs_imports:epigallocatechin_gallate'].to_csv('../output_files/EGCG_negated_edges_20220419.csv', index=False)

#### ECG

In [116]:
edges = nx_graph.get_edge_data(obodict['ECG'][0], obodict['CYP2E1'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['ECG'][0], obodict['CYP2E1'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

In [120]:
edges = nx_graph.get_edge_data(obodict['ECG'][0], obodict['UGT1A8'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['ECG'][0], obodict['UGT1A8'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

In [127]:
edges = nx_graph.get_edge_data(obodict['ECG'][0], obo.PR_P78381)
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

directly negatively regulates activity of
{'predicate_key': 'cc72b22e0562a312256062d8d489ae8e', 'weight': 0.0, 'pmid': '29467215', 'timestamp': '2018 May', 'source_graph': 'machine_read', 'belief': 0.65}


In [171]:
edges = nx_graph.get_edge_data(obodict['ECG'][0], obodict['PGP'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['ECG'][0], obodict['PGP'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['ECG'][0], obodict['PGP'][2])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['ECG'][0], obodict['PGP'][3])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

In [121]:
get_neighbors_filtered(nx_graph, obodict['ECG'][0])

[['(-)-epicatechin-3-O-gallate',
  {'entity_type': 'RELATIONS', 'label': 'Activation'},
  'mitoxantrone',
  'machine_read',
  '2017 May'],
 ['(-)-epicatechin-3-O-gallate',
  'directly negatively regulates activity of',
  'episodic kinesigenic dyskinesia 1',
  'machine_read',
  '2019 Dec'],
 ['(-)-epicatechin-3-O-gallate',
  {'entity_type': 'RELATIONS', 'label': 'Activation'},
  'episodic kinesigenic dyskinesia 1',
  'machine_read',
  '2019 Dec'],
 ['(-)-epicatechin-3-O-gallate',
  {'entity_type': 'RELATIONS', 'label': 'Activation'},
  'transport',
  'machine_read',
  '2018 Dec 21'],
 ['(-)-epicatechin-3-O-gallate',
  'directly negatively regulates activity of',
  'transport',
  'machine_read',
  '2000 Nov'],
 ['(-)-epicatechin-3-O-gallate',
  'directly negatively regulates activity of',
  'SLCO1A2_gene',
  'machine_read',
  '2014 Apr'],
 ['(-)-epicatechin-3-O-gallate',
  'directly negatively regulates activity of',
  'sulfate',
  'machine_read',
  '2011 May'],
 ['(-)-epicatechin-3-O-ga

In [172]:
df_neg.loc[df_neg['subject_obo'] == 'CHEBI_70255']

,pmid,subject_cui,subject_type,object_cui,object_type,year,subject_name,predicate,object_name,source_text,Unnamed: 10,predicate_obo,subject_obo,object_obo
1047,22969825,C0059422,"orch,phsu",C1148926,moft,2012.0,Epicatechin-3-Gallate,neg_disrupts,adenosinetriphosphatase activity,"22969825_ascii.txt.tx.1 Of these, sennoside A,...",NaN,NaN,CHEBI_70255,NaN
1137,14647974,C0059422,"orch,phsu",C0596235,"elii,phsu",2004.0,Epicatechin-3-Gallate,neg_interacts_with,Calcium ion,14647974_ascii.txt.tx.1 EGCG has no effects on...,NaN,NaN,CHEBI_70255,CHEBI_39124


#### Epicatechin

In [136]:
edges = nx_graph.get_edge_data(obodict['epicatechin'][0], obodict['CYP1A1'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['epicatechin'][0], obodict['CYP1A1'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

directly negatively regulates activity of
{'predicate_key': '9284bde1b3f4fbdf0a5d44c3211e50c0', 'weight': 0.0, 'pmid': '20583320', 'timestamp': '2011.0', 'source_graph': 'machine_read', 'belief': 0.8}


In [140]:
edges = nx_graph.get_edge_data(obodict['epicatechin'][0], obodict['UGT1A8'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['epicatechin'][0], obodict['UGT1A8'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

In [173]:
df_neg.loc[df_neg['subject_obo'] == 'CHEBI_90']

,pmid,subject_cui,subject_type,object_cui,object_type,year,subject_name,predicate,object_name,source_text,Unnamed: 10,predicate_obo,subject_obo,object_obo
110,29193528,C0014485,"orch,phsu",C0596235,"elii,phsu",2018.0,Epicatechin,neg_interacts_with,Calcium ion,"29193528_ascii.txt.tx.2 Therefore, the follow-...",NaN,NaN,CHEBI_90,CHEBI_39124
111,29193528,C0014485,"orch,phsu",C3543842,phsu,2018.0,Epicatechin,neg_interacts_with,TONICS,29193528_ascii.txt.tx.1 3 | RESULTS 3.1 | The ...,NaN,NaN,CHEBI_90,NaN
190,22015496,C0014485,"orch,phsu",C0030055,clna,2011.0,Epicatechin,neg_affects,Oxygen Consumption,"22015496_ascii.txt.tx.1 Epicatechin, quercetin...",NaN,NaN,CHEBI_90,NaN
243,29205863,C0014485,"orch,phsu",C0174599,"aapp,gngm,bacs",2018.0,Epicatechin,neg_interacts_with,"SLC2A2 protein, human","29205863_ascii.txt.tx.3 Interestingly, EC an...",NaN,NaN,CHEBI_90,PR_P11168
244,29205863,C0014485,"orch,phsu",C1420201,"gngm,aapp",2018.0,Epicatechin,neg_interacts_with,SLC5A2 gene,"29205863_ascii.txt.tx.3 Interestingly, EC an...",NaN,NaN,CHEBI_90,PR_000015169
285,16978655,C0014485,"orch,phsu",C0007620,celf,2006.0,Epicatechin,neg_affects,Cell Survival,16978655_ascii.txt.tx.3 Catechin and epicatech...,NaN,NaN,CHEBI_90,NaN
624,17077187,C0014485,"orch,phsu",C0761993,"aapp,gngm,bacs",2007.0,Epicatechin,neg_stimulates,"ABCG2 protein, human",17077187_ascii.txt.tx.1 Monofunctional inducer...,NaN,NaN,CHEBI_90,PR_Q9UNQ0
672,23252598,C0014485,"orch,phsu",C0034693,mamm,2013.0,Epicatechin,neg_administered_to,Rattus norvegicus,23252598_ascii.txt.tx.2 Capillarity was quanti...,NaN,NaN,CHEBI_90,NCBITaxon_10116
673,23252598,C0014485,"orch,phsu",C2699239,mamm,2013.0,Epicatechin,neg_administered_to,"SD, Rat Strain",23252598_ascii.txt.tx.2 Capillarity was quanti...,NaN,NaN,CHEBI_90,NaN
675,23252598,C0014485,"orch,phsu",C0429627,fndg,2013.0,Epicatechin,neg_augments,Oxygen uptake,23252598_ascii.txt.tx.2 These results suggest ...,NaN,NaN,CHEBI_90,NaN


In [141]:
get_neighbors_filtered(nx_graph,obodict['epicatechin'][0])

[['(-)-epicatechin',
  {'entity_type': 'RELATIONS', 'label': 'Activation'},
  'Peroxidase',
  'machine_read',
  '2011 Dec'],
 ['(-)-epicatechin',
  'directly negatively regulates activity of',
  'Cytochromes_c',
  'machine_read',
  '2011 Dec'],
 ['(-)-epicatechin',
  {'entity_type': 'RELATIONS', 'label': 'Activation'},
  'Cytochromes_c',
  'machine_read',
  '2011 Dec'],
 ['(-)-epicatechin',
  {'entity_type': 'RELATIONS', 'label': 'Activation'},
  'time',
  'machine_read',
  '2015 Jan'],
 ['(-)-epicatechin',
  'directly negatively regulates activity of',
  'ubiquinone-1',
  'machine_read',
  '2011 Dec'],
 ['(-)-epicatechin',
  {'entity_type': 'RELATIONS', 'label': 'Activation'},
  'transport',
  'machine_read',
  '2008 Feb'],
 ['(-)-epicatechin',
  {'entity_type': 'RELATIONS', 'label': 'Activation'},
  'N(gamma)-nitro-L-arginine methyl ester',
  'machine_read',
  '1999 Apr 19'],
 ['(-)-epicatechin',
  'positively regulates',
  'N(gamma)-nitro-L-arginine methyl ester',
  'machine_read',


In [144]:
#enantiomer
get_neighbors_filtered(nx_graph, obo.CHEBI_76125)

[]

#### Epigallocatechin

In [182]:
edges = nx_graph.get_edge_data(obodict['epigallocatechin'][0], obodict['UGT'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['epigallocatechin'][0], obodict['UGT'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

In [176]:
df_neg.loc[df_neg['subject_obo'] == 'CHEBI_42255']

,pmid,subject_cui,subject_type,object_cui,object_type,year,subject_name,predicate,object_name,source_text,Unnamed: 10,predicate_obo,subject_obo,object_obo
456,19111536,C2348459,orch,C0020517,patf,2009.0,Epigallocatechin,neg_treats,Hypersensitivity,19111536_ascii.txt.tx.1 The apparent dissociat...,NaN,NaN,CHEBI_42255,NaN
518,15277314,C2348459,orch,C1519726,moft,2004.0,Epigallocatechin,neg_disrupts,Tyrosine Phosphorylation,15277314_ascii.txt.tx.1 Activation of is contr...,NaN,NaN,CHEBI_42255,NaN
857,15729621,C2348459,orch,C1412077,"gngm,aapp",2005.0,Epigallocatechin,neg_interacts_with,ABCC2 gene,15729621_ascii.txt.tx.1 Func- tional assays in...,NaN,NaN,CHEBI_42255,PR_000003558
1215,28346686,C2348459,orch,C4055506,fndg,2017.0,Epigallocatechin,neg_augments,Accumulation,"28346686_ascii.txt.tx.1 In contrast, other stu...",NaN,NaN,CHEBI_42255,NaN
1502,15277314,C2348459,orch,C1519726,moft,2004.0,Epigallocatechin,neg_disrupts,Tyrosine Phosphorylation,15277314_ascii.txt.tx.1|relation|C1120843|mito...,NaN,NaN,CHEBI_42255,NaN
1648,28346686,C2348459,orch,C4055506,fndg,2017.0,Epigallocatechin,neg_augments,Accumulation,28346686_ascii.txt.tx.4 Some studies reported ...,NaN,NaN,CHEBI_42255,NaN


In [183]:
get_neighbors_filtered(nx_graph, obodict['epigallocatechin'][0])

[['(-)-epigallocatechin',
  'positively regulates',
  'stachyose',
  'machine_read',
  '2016.0'],
 ['(-)-epigallocatechin',
  'negatively regulates',
  'import into cell',
  'machine_read',
  '2011.0'],
 ['(-)-epigallocatechin',
  'directly negatively regulates activity of',
  'potassium atom',
  'machine_read',
  '2007.0'],
 ['(-)-epigallocatechin',
  'interacts with',
  '(-)-epigallocatechin gallate',
  'machine_read_INF',
  '2013.0'],
 ['(-)-epigallocatechin',
  {'entity_type': 'RELATIONS', 'label': 'affects'},
  'blood vessel',
  'machine_read',
  '2003.0'],
 ['(-)-epigallocatechin',
  'interacts with',
  'voltage-gated potassium channel subunit KCNQ5',
  'machine_read_INF',
  '2021.0'],
 ['(-)-epigallocatechin',
  'interacts with',
  'procyanidin B2',
  'machine_read_INF',
  '2020.0'],
 ['(-)-epigallocatechin',
  'positively regulates',
  'catechin',
  'machine_read_INF']]

#### Catechin

In [188]:
edges = nx_graph.get_edge_data(obodict['catechin'][0], obodict['UGT1A8'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['catechin'][0], obodict['UGT1A8'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['catechin'][1], obodict['UGT1A8'][0])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])
edges = nx_graph.get_edge_data(obodict['catechin'][1], obodict['UGT1A8'][1])
if edges:
    for item in edges:
        print(nodeLabels[str(item)])
        print(edges[item])

In [190]:
df_neg.loc[df_neg['subject_obo'] == 'CHEBI_23053']

,pmid,subject_cui,subject_type,object_cui,object_type,year,subject_name,predicate,object_name,source_text,Unnamed: 10,predicate_obo,subject_obo,object_obo
1,12659723,C0007404,"orch,phsu",C1420188,"gngm,aapp",2003.0,Catechin,neg_interacts_with,SLC35A2 gene,12659723_ascii.txt.tx.2 Lhoste et al./ Food an...,NaN,NaN,CHEBI_23053,PR_000015090
17,20471964,C0007404,"orch,phsu",C0243144,phsf,2010.0,Catechin,neg_disrupts,Uptake,20471964_ascii.txt.tx.2 These results further ...,NaN,NaN,CHEBI_23053,GO_0098657
76,15104253,C0007404,"orch,phsu",C0537026,"gngm,aapp",2004.0,Catechin,neg_stimulates(spec),UGT1A1 gene,"15104253_ascii.txt.tx.2 Here, we investigated ...",NaN,NaN,CHEBI_23053,PR_000017048
88,21776468,C0007404,"orch,phsu",C0166441,"aapp,gngm,bacs",2010.0,Catechin,neg_interacts_with,Glucose Transporter Type 4,21776468_ascii.txt.tx.2 This is the first repo...,NaN,NaN,CHEBI_23053,PR_000013058
131,18936213,C0007404,"orch,phsu",C3842459,fndg,2008.0,Catechin,neg_affects,Energetic,18936213_ascii.txt.tx.1 From modulation of foo...,NaN,NaN,CHEBI_23053,NaN
133,18936213,C0007404,"orch,phsu",C0014939,"horm,orch,phsu",2008.0,Catechin,neg_interacts_with,Estrogens,18936213_ascii.txt.tx.1 In that experimental s...,NaN,NaN,CHEBI_23053,CHEBI_50114
172,15041478,C0007404,"orch,phsu",C0380168,"aapp,gngm,bacs",2004.0,Catechin,neg_inhibits,glutathione S-conjugate export pump,15041478_ascii.txt.tx.1 Flavonoids without a C...,NaN,NaN,CHEBI_23053,NaN
220,27374085,C0007404,"orch,phsu",C1538577,"gngm,aapp",2016.0,Catechin,neg_stimulates,PARP1 gene,27374085_ascii.txt.tx.1 A significant increase...,NaN,NaN,CHEBI_23053,PR_000012289
221,27374085,C0007404,"orch,phsu",C0020517,patf,2016.0,Catechin,neg_affects,Hypersensitivity,27374085_ascii.txt.tx.3 Neuroblastoma cells wi...,NaN,NaN,CHEBI_23053,NaN
223,27374085,C0007404,"orch,phsu",C0027819,neop,2016.0,Catechin,neg_treats,Neuroblastoma,27374085_ascii.txt.tx.1 To determine whether C...,NaN,NaN,CHEBI_23053,NaN


In [196]:
len(get_neighbors_filtered(nx_graph, obodict['catechin'][0]))

1023

In [197]:
len(get_neighbors_filtered(nx_graph, obodict['catechin'][1]))

502

#### Gallocatechin

In [201]:
get_neighbors(nx_graph, obodict['gallocatechin'][1])

[['gallocatechin',
  'directly negatively regulates activity of',
  'Tea',
  'machine_read',
  '2014'],
 ['gallocatechin',
  'existence overlaps',
  'flavonoids',
  'machine_read',
  '2016.0'],
 ['gallocatechin', 'interacts with', 'chelator', 'machine_read_INF', '2011.0'],
 ['gallocatechin',
  'positively regulates',
  'very-long-chain enoyl-CoA reductase',
  'machine_read',
  '2004.0'],
 ['gallocatechin', 'interacts with', 'interleukin-6 (human)'],
 ['gallocatechin', 'subClassOf', 'chemical entity'],
 ['gallocatechin', 'has role', 'metabolite'],
 ['gallocatechin', 'interacts with', 'transcription factor p65 (human)'],
 ['gallocatechin', 'subClassOf', 'catechin'],
 ['gallocatechin', 'subClassOf', "flavan-3,3',4',5,5',7-hexol"],
 ['gallocatechin',
  'interacts with',
  'glutathione reductase, mitochondrial (human)'],
 ['gallocatechin', 'interacts with', 'Toll-like receptor 4 (human)'],
 ['gallocatechin', 'interacts with', 'myeloperoxidase (human)'],
 ['gallocatechin', 'interacts with', 

In [207]:
df_neg.loc[df_neg['object_name'] == 'gallocatechin']

,pmid,subject_cui,subject_type,object_cui,object_type,year,subject_name,predicate,object_name,source_text,Unnamed: 10,predicate_obo,subject_obo,object_obo
